1. Khai báo thư viện và imports

In [2]:
#!/usr/bin/env python3
import argparse
import sys
import time
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import pad, unpad

Hàm encrypt_decrypt_aes

In [3]:
def encrypt_decrypt_aes(plaintext: bytes, key: bytes, iterations: int = 10):
    """Mã hóa và giải mã AES nhiều lần, trả về thời gian trung bình (encrypt, decrypt)."""
    enc_times, dec_times = [], []
    for _ in range(iterations):
        iv = get_random_bytes(16)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        # encrypt
        t0 = time.perf_counter()
        ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))
        t1 = time.perf_counter()
        # decrypt
        cipher2 = AES.new(key, AES.MODE_CBC, iv)
        _ = unpad(cipher2.decrypt(ciphertext), AES.block_size)
        t2 = time.perf_counter()
        enc_times.append(t1 - t0)
        dec_times.append(t2 - t1)
    return sum(enc_times)/iterations, sum(dec_times)/iterations

2. Hàm encrypt_decrypt_rsa

In [4]:
def encrypt_decrypt_rsa(sym_key: bytes, rsa_key: RSA.RsaKey, iterations: int = 10):
    """Mã hóa và giải mã khóa AES bằng RSA nhiều lần, trả về thời gian trung bình (encrypt, decrypt)."""
    pub = rsa_key.publickey()
    encryptor = PKCS1_OAEP.new(pub)
    decryptor = PKCS1_OAEP.new(rsa_key)
    enc_times, dec_times = [], []
    for _ in range(iterations):
        t0 = time.perf_counter()
        encrypted = encryptor.encrypt(sym_key)
        t1 = time.perf_counter()
        _ = decryptor.decrypt(encrypted)
        t2 = time.perf_counter()
        enc_times.append(t1 - t0)
        dec_times.append(t2 - t1)
    return sum(enc_times)/iterations, sum(dec_times)/iterations

3. Hàm main() và phân tích tham số

In [5]:
def main():
    parser = argparse.ArgumentParser(description="Demo hiệu năng AES và RSA")
    parser.add_argument("-m", "--message", type=str,
                        default="Hello, this is a test message!",
                        help="Văn bản thuần để mã hóa bằng AES")
    parser.add_argument("-i", "--iterations", type=int, default=50,
                        help="Số lần lặp để tính thời gian trung bình")
    args = parser.parse_args(args=[])

    plaintext = args.message.encode()
    aes_key = get_random_bytes(16)

    print(f"\n[*] AES: thực hiện {args.iterations} lần trên văn bản độ dài {len(plaintext)} byte")
    aes_enc_avg, aes_dec_avg = encrypt_decrypt_aes(plaintext, aes_key, args.iterations)
    print(f"    → Thời gian mã hóa trung bình: {aes_enc_avg*1e3:.3f} ms")
    print(f"    → Thời gian giải mã trung bình: {aes_dec_avg*1e3:.3f} ms")

    print("\n[*] RSA: mã hóa/giải mã khóa AES 16 byte")
    rsa_key = RSA.generate(2048)
    rsa_enc_avg, rsa_dec_avg = encrypt_decrypt_rsa(aes_key, rsa_key, args.iterations)
    print(f"    → Thời gian mã hóa RSA trung bình: {rsa_enc_avg*1e3:.3f} ms")
    print(f"    → Thời gian giải mã RSA trung bình: {rsa_dec_avg*1e3:.3f} ms")

    print("\n[*] Tóm tắt:")
    speedup_enc = rsa_enc_avg / aes_enc_avg
    speedup_dec = rsa_dec_avg / aes_dec_avg
    print(f"    • AES mã hóa nhanh hơn RSA khoảng {speedup_enc:.1f}×")
    print(f"    • AES giải mã nhanh hơn RSA khoảng {speedup_dec:.1f}×")

if __name__ == "__main__":
    main()


[*] AES: thực hiện 50 lần trên văn bản độ dài 30 byte
    → Thời gian mã hóa trung bình: 0.012 ms
    → Thời gian giải mã trung bình: 0.013 ms

[*] RSA: mã hóa/giải mã khóa AES 16 byte
    → Thời gian mã hóa RSA trung bình: 0.505 ms
    → Thời gian giải mã RSA trung bình: 3.224 ms

[*] Tóm tắt:
    • AES mã hóa nhanh hơn RSA khoảng 41.5×
    • AES giải mã nhanh hơn RSA khoảng 246.8×
